In [1]:
import pandas as pd

In [2]:
segments_df = pd.read_csv('data/segments.csv')

In [3]:
segments_df.sample(5)

,Document ID,Segment position,Text,Tags,Summary,Non-operative,Not AI-related,Segment annotated,Segment validated,Summaries and tags may include unreviewed machine output,...,"Strategies: Licensing, registration, and certification",Strategies: New institution,Strategies: Performance requirements,Strategies: Pilots and testbeds,Strategies: Tiering,Strategies: Tiering: Tiering based on domain of application,Strategies: Tiering: Tiering based on generality,Strategies: Tiering: Tiering based on impact,Strategies: Tiering: Tiering based on inputs,Strategies: Tiering: Tiering based on planning ability
7106,2050,2,"NOW, THEREFORE, I, Kay Ivey, Governor of the S...",NaN,Establishes the Governor’s Task Force on Gener...,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3751,995,31,"\nHUMAN ALTERNATIVES, CONSIDERATION, AND FALLB...","Harms: Violation of civil or human rights, inc...",Requires opt-out options of automated systems ...,False,False,True,True,False,...,False,False,False,False,True,True,False,True,False,False
6070,1782,22,8. THE ATTORNEY GENERAL MAY PROMULGATE FURT...,Strategies: Evaluation;Strategies: Evaluation:...,Allows the Attorney General to establish rules...,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3010,768,24,3.\tTransparency: We will share summaries of C...,Strategies: Governance development,Commits to additional governance practices rel...,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
5240,1390,4,(3) Carry out pilot demonstrations to improve ...,NaN,Promotes AI standardization through demonstrat...,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


#### various annotated fields which can augment the knowledge graph

1. Strategies
2. Risk
3. Incentives
4. Risk Factors
5. Harms
6. Application
7. Not AI-related

- These tags can also be parsed from **Tags** field, splitting on ; and :
- The text segments can be used instead of parsing the full text to build the knowledge graph
- The summary can additionally be used for NER and relationship extraction
- Not all the full texts may be segmented

In [4]:
documents_df = pd.read_csv("data/documents.csv")
documents_df.sample(5)

,AGORA ID,Official name,Casual name,Link to document,Authority,Collections,Most recent activity,Most recent activity date,Proposed date,Annotated?,...,"Strategies: Licensing, registration, and certification",Strategies: New institution,Strategies: Performance requirements,Strategies: Pilots and testbeds,Strategies: Tiering,Strategies: Tiering: Tiering based on domain of application,Strategies: Tiering: Tiering based on generality,Strategies: Tiering: Tiering based on impact,Strategies: Tiering: Tiering based on inputs,Strategies: Tiering: Tiering based on planning ability
325,192,National Defense Authorization Act for Fiscal ...,"FY2024 NDAA, Section 1521 (""Control and manage...",https://www.congress.gov/bill/118th-congress/h...,United States Congress,NDAA provisions;U.S. federal laws,Enacted,2023-12-22,NaN,True,...,False,True,True,False,False,False,False,False,False,False
510,888,Targeting Online Sales of Fentanyl Act,Targeting Online Sales of Fentanyl Act,https://www.congress.gov/bill/118th-congress/h...,United States Congress,U.S. federal laws,Defunct,2025-01-03,2024-03-22,False,...,False,False,False,False,False,False,False,False,False,False
497,771,"Microsoft Responsible AI Standard, v2","Microsoft Responsible AI Standard, v2",https://blogs.microsoft.com/wp-content/uploads...,Private-sector companies,Corporate policies and commitments,Enacted,2024-06-01,2024-06-01,True,...,False,False,True,False,True,False,False,True,False,False
817,365,Massachusetts Bill S. 31,"Massachusetts Bill S. 31 (""Regulate GenAI"")",https://malegislature.gov/Bills/193/SD1827,Massachusetts,U.S. state and local documents,Proposed,2023-01-20,2023-01-20,False,...,False,False,False,False,False,False,False,False,False,False
821,1045,Australia’s AI Ethics Principles,Australia’s AI Ethics Principles,https://www.industry.gov.au/publications/austr...,Government of Australia,Miscellaneous documents,Enacted,2019-11-07,2019-11-07,False,...,False,False,False,False,False,False,False,False,False,False


- Since there is an official name and also a casual name both needs to be linked since different names may be used to reference the same document
- Authority and Collections can be entities for sources
- Applies to the governement and public can be another tag to connect scope. But additional scoping tags are required to be generated.
- The long summary can be used to generate additional nodes and tags
- The same 5 areas `Application`, `Risk Factor`, `Strategies`, `Harms`, `Incentives` are included again.
- Summary and document level tags are merged into one field called Tags
- Also check `Official plaintext unavailable/infeasible`

In [5]:
authorities_df = pd.read_csv("data/authorities.csv")
authorities_df.sample(5)

,Name,Jurisdiction,Parent authority
96,Missouri,United States,State governments
2,Chinese central government,China,NaN
102,Colorado,United States,State governments
0,Private-sector companies,NaN,NaN
22,Department of Health and Human Services,United States,Federal government


- `Name`, `Jurisdiction` and `Parent authority` can be used as entites directly along with the corresponding edges to define structural relationship between an institution and a document

In [6]:
collections_df = pd.read_csv("data/collections.csv")
collections_df.sample(5)

,Name,Description
5,U.S. state and local documents,AI-related laws from U.S. states and localitie...
2,and agency policies,NaN
7,NDAA provisions,AI-related provisions from recent National Def...
12,"U.S. regulations, executive orders, and agency...",U.S. federal regulations and other AI-related ...
0,executive orders,NaN


- collections can be used as broad topics, and more can be added based on other methods such as LDA